In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from IPython.core.display_functions import display
from biogeme.expressions import Beta, Variable
from biogeme.models import loglogit
from biogeme.segmentation import DiscreteSegmentationTuple, segmented_beta

In [2]:
data = pd.read_csv("lpmc01.dat", sep = '\t')

In [3]:
data.describe()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,40272.741000,8650.721200,0.770400,1.537200,3.058600,2.890400,2.818800,2.21460,0.660400,1.975000,...,0.158076,0.091207,0.173027,0.044790,0.370800,0.284051,1.592746,0.838318,1.13400,0.336089
std,23395.859181,5067.720038,1.027178,1.771565,1.079907,1.185018,2.232614,1.72486,0.467135,0.817624,...,0.090550,0.176136,0.190790,0.078831,0.622402,0.253053,1.523202,0.827559,3.25932,0.200525
min,12.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.00000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.009722,0.000000,0.030000,0.00000,0.000000
25%,19902.500000,4214.000000,0.000000,0.000000,3.000000,2.000000,1.000000,1.00000,0.000000,1.000000,...,0.092222,0.000000,0.036111,0.000000,0.000000,0.107778,0.000000,0.280000,0.00000,0.172961
50%,40351.500000,8630.500000,0.000000,1.000000,3.000000,3.000000,2.000000,1.00000,1.000000,2.000000,...,0.142222,0.000000,0.111667,0.000000,0.000000,0.189583,1.500000,0.540000,0.00000,0.303972
75%,60283.750000,12922.000000,1.000000,2.000000,4.000000,3.000000,6.000000,4.00000,1.000000,3.000000,...,0.206389,0.100000,0.250278,0.091667,1.000000,0.377014,2.400000,1.112500,0.00000,0.487060
max,81084.000000,17615.000000,7.000000,14.000000,4.000000,5.000000,6.000000,5.00000,1.000000,3.000000,...,0.804444,1.183333,1.871944,0.750000,4.000000,1.730556,10.600000,9.220000,10.50000,0.957627


## Variable Definition

In [4]:
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access') # Predicted total access and egress time for public transport route in hours
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int') # Time taken (hrs) at each interchange point
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')   # Estimated fuel cost of driving route in GBP
cost_driving_ccharge = Variable('cost_driving_ccharge')  # Estimated congestion charge cost of driving route in GBP
driving_traffic_percent = Variable('driving_traffic_percent')

In [5]:
# Define driving cost
cost_driving = cost_driving_ccharge + cost_driving_fuel

# Define time taken by each mode of transport
time_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int 

time_cycling = dur_cycling
time_walking = dur_walking
time_driving = dur_driving

# Model 0

In [6]:
# Define ASCs 
asc_car = Beta(name='asc_car', value=0, lowerbound=None, upperbound=None, status=0)
asc_walk = Beta(name='asc_walk', value=0, lowerbound=None, upperbound=None, status=0)
asc_pt = Beta(name='asc_pt', value=0, lowerbound=None, upperbound=None, status=0)
